# Introduction

The algorithm is broken down into multiple created functions, where the following order counts: <br /> <br /> 
$\;\;\;\;\;\;\;\;\;\;\;\;\;$ 1) $\;\;\;\;\;$Convert matrix into a system of coordinates and remove the wall coordinates <br /> 
$\;\;\;\;\;\;\;\;\;\;\;\;\;$ 2) $\;\;\;\;\;$First attempts to convert the list of coordinates to local areas <br /> 
$\;\;\;\;\;\;\;\;\;\;\;\;\;$ 3) $\;\;\;\;\;$Repeat the searching process untill the local areas are detected (one optimization might not be sufficient) <br /> 
$\;\;\;\;\;\;\;\;\;\;\;\;\;$ 4) $\;\;\;\;\;$Detect the number of required cleaning sessions <br /> 

# 1) Convert matrix into a system of coordinates and remove the wall coordinates

### $\;\;\;\;\;\;\;\;\;\;\;$Detect whether the coordinate corresponds to a wall

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$The inputs are: <br />
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ $-$ The matrix X <br />
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ $-$ The i (= row) coordinate <br />
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ $-$ The j (= column) coordinate

In [108]:
def is_wall(X, i, j):
    single_cell = X[i][j]
                    
    # If wall, return True
    if single_cell == '#':
        return True
        
    else:
        return False

#### $\;\;\;\;\;\;\;\;\;\;\;$ Convert the matrix into list of coordinates (i = row, j = column)

$\;\;\;\;\;\;\;\;\;\;\;\;\;$ Each position in the matrix has an associated coordinate, with the first cell on the top left being coordinate (0, 0).  <br /> 
$\;\;\;\;\;\;\;\;\;\;\;\;\;$ The bottom right cell is coordinate (n-1, n-1), where n is the length of the row/columns (1 is substracted from n since the index in Python starts at 0 and not 1).  <br />$\;\;\;\;\;\;\;\;\;\;\;\;\;$ For simplicity, the assumtion is made that the number of rows equals the numer of columns:<br /><br />   <big>$$ n_{i} = n_{j} $$</big>  <br />

In [3]:
def create_coordinates(X):

    n_rows = len(X)
    n_cols = len(X[0])  # The assumtion is made that the length is equal for each row
                    
    coordinates = []
        
    for i in range(n_rows):
        for j in range(n_cols):
            coordinates.append((i, j))
 
    return coordinates

# 2) First attempts to convert the list of coordinates to local areas 


### $\;\;\;\;\;\;\;\;\;\;\;$ Flatten a list

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ In the Numpy package there is a specific function that flatten a Numpy array. <br /> $\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ Since I do not use Numpy arrays (but rather lists) in this notebook, I prefer to write a clean and readable function to flatten a list, <br /> $\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ even though this can also be performed by doing the following: sum(your_list, [].
Since I know the list will contain lists (which will contain tuples), 
 <br /> $\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$  the goal is to get one list of tuples. For example: <br /> <br />
 $\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$  $$[ \; \;[ \; \;(0, 0),\;(0, 1) \;\;],  \;[ \; \;(2, 3) \; \;] \;\;]  \;\;\;\;  -> \;\;\;\;    [ \; \;(0, 0), \;(0, 1), \;(2, 3) \; \;]$$ <br />

In [4]:
def flatten_list(lst):
    
    flattened_list = []
    
    for inner_list in lst:
        for tuple_ in inner_list:
            flattened_list.append(tuple_)
    
    return flattened_list

### $\;\;\;\;\;\;\;\;\;\;\;$ Delete the coordinates which are walls

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ We are only interested in neighbouring rooms and using this method, it is not necessary to keep the coordinates of the walls (hence, we delete them).  

In [5]:
def clean_coordinates(X):

    coordinates = create_coordinates(X)
    cors_with_no_walls = []
    
    
    for cor in coordinates:
        i, j = cor
        
        if is_wall(i, j, X):
            continue
            
        else:
            cors_with_no_walls.append(cor)
    
    return cors_with_no_walls

### $\;\;\;\;\;\;\;\;\;\; \;$ Find local areas 

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ Fiding all local araes might take more than one search using this method, depending on the complexity (or smoothness) of the local areas. 
<br />
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ For example, a local area consisting of only a horizonal line (start: i, j , end: i+x, j), will be found in one search. Larger local areas (certainly in case <br /> $\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ there is some interference of walls) may take 2 or more searched. 

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ To find the first search, the following code is used:

In [99]:
def find_local_areas(X):

    clean_cors = clean_coordinates(X)

    local_areas = []

    for cor in clean_cors:
        
        # If the coordinate of one of the local_aras is already in the flattened local_areas list, continue to the next    
        if cor in flatten_list(local_areas): 
            continue
            
        i, j = cor
        local =[cor]
        latest_i, latest_j = cor
        
        for cor2 in clean_cors:
            
            # If the coordinate of the inner loop is in the flattened local_areas list, continue to the next    
            if cor2 in flatten_list(local_areas): 
                continue
                
            i2, j2 = cor2 
            
            # If the absolute difference between both coordinates equals 1, it is sure that the rooms are neighbouring  
            difference = abs(i2 - latest_i) + abs(j2 - latest_j)
            
            if difference == 1:
                local.append(cor2)
                latest_i, latest_j = cor2
        
        local_areas.append(local)

    return local_areas

# 3) Repeat the searching process untill the local areas are detected (one optimization might not be sufficient)

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ In the below, the first search is tested. In case there are still 2 areas which should be combined, it will cluster the 2 areas into 1 local area.

In [92]:
def clusterize(X):
    
    local_areas = find_local_areas(X)
    new_cluster = []
    removed = []

    # Initilize to True to make the while loop work
    any_change = True
    count = 0

    while any_change:
        
        # Re-initiliaze to false, and in case any movement appears set to True
        any_change = False
    
        if count > 0:
            local_areas = new_cluster
            new_cluster = []
            removed = []
            
        for lst in local_areas:

            if lst[0] in removed:
                continue

            match = False

            removed.extend(lst)

            for lst2 in local_areas:

                if lst2[0] in removed:
                    continue

                for cor in lst:
                    i, j = cor

                    for cor2 in lst2:
                        i2, j2 = cor2
                        difference = abs(i2 - i) + abs(j2 - j)

                    if difference == 1:
                        add = lst + lst2
                        removed.extend(lst2)
                        new_cluster.append(add)
                        match = True
                        any_change = True
                        break

            if match == False:
                new_cluster.append(lst)
                removed.extend(lst)
        
        count =+ 1

    return new_cluster

# 4) Detect the number of required cleaning sessions

### $\;\;\;\;\;\;\;\;\;\;\;$ Detect whether there is a dirty room in the local areas

In [60]:
def is_dirty(i, j, X):
    single_position = X[i][j]
    
    if single_position == '1':
        return True
    
    else:
        return False

### $\;\;\;\;\;\;\;\;\;\;\;$ Iterate over local areas and count the required clean sessions

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ Iterate over the rooms in each local area. If a local area contains at least one dirty room, add 1 to the number of required cleaning sessions and continue to the next local area

In [109]:
def n_cleans_required(X):
    
    local_areas = clusterize(X)
    n_cleans = 0
    
    # Iterate over each local area
    for local_area in local_areas:
        dirty_room = False
        
        # Iterate over the rooms in the lcoal area
        for rooms_cor in local_area: 
            i, j = rooms_cor
            
            '''In case there is a dirty room in a local alrea, add 
                one cleaning session and go to the next local area 
                (by breaking the inner loop)'''
            if is_dirty(i, j, X):
                dirty_room = True
                n_cleans += 1
                break
        
    return 'Number of local areas: ', len(local_areas), '\nNumber of cleans required: ', n_cleans